## 构造模型
### 继承Module构建模型

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层 这里是两个全连接层
    def __init__(self, **kwargs):   # 重写init方法
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256)   # 添加隐藏层
        self.act = nn.ReLU()                # 激活函数
        self.output = nn.Linear(256, 10)    # 输出层

    # 重写前向计算
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [2]:
X = torch.rand(2, 784)
net = MLP() 
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0291,  0.1236,  0.1575, -0.0775,  0.1388, -0.0207,  0.0736,  0.0382,
          0.0693, -0.1443],
        [ 0.0658,  0.1074,  0.1143,  0.0444,  0.0876, -0.1488, -0.0525,  0.0597,
          0.1048, -0.1135]], grad_fn=<AddmmBackward0>)

### Module子类
#### Sequentital类
下面我们实现一个与Sequential类有相同功能的MySequential类。这或许可以帮助读者更加清晰地理解Sequential类的工作机制。

In [4]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        # 如果传入OrderedDict
        if len(args)==1 and isinstance(args[0], OrderedDict):
            for key, module in args[0].items():
                # add_module方法会将module添加进self._modules(一个OrderedDict)
                self.add_module(key, module)
        else:   # 如果传入的是一些module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            input = module(input)
        return input


我们用MySequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算。

In [5]:
net = MySequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0142,  0.0628, -0.2660,  0.2819, -0.0818, -0.0891, -0.1679,  0.0602,
          0.3058,  0.1863],
        [-0.0190, -0.0413, -0.1987,  0.1918,  0.0100, -0.1111, -0.2944,  0.1756,
          0.2174,  0.1130]], grad_fn=<AddmmBackward0>)